In [2]:
class LightBeam:
    
    def __init__(self, position, direction):
        self._position = position
        self._direction = direction
    
    def getPosition(self):
        return self._position
    
    def setPosition(self, position):
        self._position = position
        
    def getDirection(self):
        return self._direction
    
    def setDirection(self, direction):
        self._direction = direction

In [3]:
class MirrorGrid:
    
    def __init__(self, text):
        self._map = text.split('\n')
        self._yBound = len(self._map)
        self._xBound = len(self._map[0])
        self._lightBeams = self.initLightBeams()
        self._directionMirrorMap = self.initDirectionMirrorMap()
        self._directionToMoveMap = self.initDirectionToMoveMap()
        self._energized = {}
        self._energizedCount = 0
        
    def getEnergizedCount(self):
        return self._energizedCount
    
    def energizeGrid(self):
        while len(self._lightBeams) > 0:
            self.advanceLightBeams()
    
    def advanceLightBeams(self):
        out = []
        for lightBeam in self._lightBeams:
            out += self.advanceLightBeam(lightBeam)
        self._lightBeams = out
        
    def advanceLightBeam(self, lightBeam):
        y, x = lightBeam.getPosition()
        direction = lightBeam.getDirection()
        dy, dx = self._directionToMoveMap[direction]
        y += dy
        x += dx
        if y < 0 or y >= self._yBound or x < 0 or x >= self._xBound:
            return []
        try:
            mirrorType = self._map[y][x]
        except:
            raise Exception("Tried to access {0} {1} from direction {2}".format(y, x, direction))
        position = (y, x)
        if not (direction, mirrorType) in self._directionMirrorMap:
            if position in self._energized:
                if direction in self._energized[position]:
                    return []
                else:
                    self._energized[position].append(direction)
            else:
                self._energized[position] = [direction]
                self._energizedCount += 1
            lightBeam.setPosition(position) ## direction stays the same
            return [lightBeam]
        directions = self._directionMirrorMap[(direction, mirrorType)]
        lightBeams = []
        for postDirection in directions:
            if (y, x) in self._energized:
                if postDirection in self._energized[(y,x)]:
                    continue
                else:
                    self._energized[(y, x)].append(postDirection)
                    lightBeams.append(LightBeam((y, x), postDirection))
            else:
                self._energized[(y, x)] = [postDirection]
                self._energizedCount += 1
                lightBeams.append(LightBeam((y, x), postDirection))
        return lightBeams
    
    def printGrid(self):
        for i in range(self._yBound):
            line = ""
            for j in range(self._xBound):
                if (i, j) in self._energized:
                    line += '#'
                else:
                    line += '.'
            print(line)
        
    def initLightBeams(self):
        return [LightBeam((0, -1), '>')]
    
    def initDirectionMirrorMap(self):
        map_ = {}
        map_[('>','/')] = ['^']
        map_[('>','\\')] = ['v']
        map_[('>','|')] = ['v', '^']
        map_[('<','/')] = ['v']
        map_[('<','\\')] = ['^']
        map_[('<','|')] = ['v', '^']
        map_[('^','/')] = ['>']
        map_[('^','\\')] = ['<']
        map_[('^','-')] = ['<', '>']
        map_[('v','/')] = ['<']
        map_[('v','\\')] = ['>']
        map_[('v','-')] = ['<', '>']
        return map_
    
    def initDirectionToMoveMap(self):
        map_ = {}
        map_['>'] = (0, 1)
        map_['<'] = (0, -1)
        map_['^'] = (-1, 0)
        map_['v'] = (1, 0)
        return map_

In [4]:
mirrorGrid = MirrorGrid(text)
mirrorGrid.energizeGrid()
print(mirrorGrid.getEnergizedCount())

7477


In [5]:
class MaxEnergyMirrorGrid(MirrorGrid):
    
    def getMaximumEnergizedCount(self):
        max_ = 0
        max_ = self.maxEnergizedCountColumn(-1, '>', max_)
        max_ = self.maxEnergizedCountColumn(self._xBound, '<', max_)
        max_ = self.maxEnergizedCountRow(-1, 'v', max_)
        max_ = self.maxEnergizedCountRow(self._yBound, '^', max_)
        return max_
        
    def maxEnergizedCountColumn(self, column, direction, max_):
        for i in range(self._yBound):
            current = self.initAndEnergizeGrid((i, column), direction)
            if current > max_:
                max_ = current
        return max_
    
    def initAndEnergizeGrid(self, position, direction):
        self.resetGrid()
        self.resetInitLightBeams(position, direction)
        self.energizeGrid()
        return self.getEnergizedCount()
    
    def resetGrid(self):
        self._energized = {}
        self._energizedCount = 0
    
    def resetInitLightBeams(self, position, direction):
        self._lightBeams = [LightBeam(position, direction)]
    
    def maxEnergizedCountRow(self, row, direction, max_):
        for i in range(self._xBound):
            current = self.initAndEnergizeGrid((row, i), direction)
            if current > max_:
                max_ = current
        return max_
        

In [6]:
maxEnergyMirrorGrid = MaxEnergyMirrorGrid(text)
print(maxEnergyMirrorGrid.getMaximumEnergizedCount())

7853
